# Importing libraries

In [2]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Load dataset

In [3]:

data = pd.read_csv("../notebooks/output/subset_100k.csv")

# Data preprocessing

In [4]:
data = data[['title', 'ingredients', 'directions', 'NER', 'link']]
data.dropna(inplace=True)
data['combined_text'] = data['ingredients'] + ' ' + data['directions']
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# TF-IDF Vectorization

In [5]:
vectorizer = TfidfVectorizer(stop_words='english')
data['processed_text'] = data['combined_text'].apply(preprocess_text)
tfidf_matrix = vectorizer.fit_transform(data['processed_text'])

# Recommendation using cosine similarity

In [6]:
def recommend_recipe(user_preferences, num_recommendations=5):
    user_combined_text = ' '.join(user_preferences['available_ingredients'])
    if user_preferences['dietary_restrictions']:
        user_combined_text += ' ' + ' '.join(user_preferences['dietary_restrictions'])
    
    if user_preferences['cuisine_preferences']:
        user_combined_text += ' ' + ' '.join(user_preferences['cuisine_preferences'])
    
    if user_preferences['meal_type']:
        user_combined_text += ' ' + user_preferences['meal_type']
    
    if user_preferences['time_to_cook']:
        user_combined_text += f" {user_preferences['time_to_cook']} minutes"
    
    user_query_vector = vectorizer.transform([preprocess_text(user_combined_text)])
    
    cosine_sim = cosine_similarity(user_query_vector, tfidf_matrix)
    
    top_indices = cosine_sim[0].argsort()[-num_recommendations:][::-1]
    recommendations = data.iloc[top_indices].copy()
    
    if 'vegetarian' in user_preferences['dietary_restrictions']:
        non_veg_keywords = ['chicken', 'beef', 'pork', 'fish', 'sausage', 'bacon', 'prawn', 
                            'shrimp', 'duck', 'lamb', 'turkey', 'hamburger', 'meat']
        recommendations = recommendations[~recommendations['ingredients']
                                           .str.contains('|'.join(non_veg_keywords), case=False, na=False)]
    
    if 'gluten-free' in user_preferences['dietary_restrictions']:
        gluten_keywords = ['wheat', 'barley', 'rye', 'bread', 'pasta']
        recommendations = recommendations[~recommendations['ingredients']
                                           .str.contains('|'.join(gluten_keywords), case=False, na=False)]
    
    if 'dairy-free' in user_preferences['dietary_restrictions']:
        dairy_keywords = ['milk', 'cheese', 'butter', 'cream', 'yogurt']
        recommendations = recommendations[~recommendations['ingredients']
                                           .str.contains('|'.join(dairy_keywords), case=False, na=False)]
    
    if 'nut-free' in user_preferences['dietary_restrictions']:
        nut_keywords = ['almonds', 'cashews', 'peanuts', 'walnuts', 'hazelnuts', 
                        'pecans', 'brazil nuts', 'macadamia', 'nuts']
        recommendations = recommendations[~recommendations['ingredients']
                                           .str.contains('|'.join(nut_keywords), case=False, na=False)]
    
    return recommendations[['title', 'ingredients', 'directions', 'link']]



# Example Query 1

In [7]:
user_preferences = {
    "dietary_restrictions": ["vegetarian"],  
    "cuisine_preferences": ["Indian"],       
    "available_ingredients": ["tomato", "onion", "garlic"],  
    "time_to_cook": 30,                      
    "difficulty_level": "Easy",              
    "meal_type": "Lunch"                     
}

recommendations = recommend_recipe(user_preferences)

print("Recommended Recipes:")
for index, recipe in recommendations.iterrows():
    print(f"\nTitle: {recipe['title']}")
    print(f"Ingredients: {recipe['ingredients']}")
    print(f"Directions: {recipe['directions']}")
    print(f"Link: {recipe['link']}")


Recommended Recipes:

Title: Chutney Glaze
Ingredients: ["2 Indian mangoes, peeled and chopped", "1 c. honey", "1/2 to 1 c. pineapple juice", "1 tsp. ground cloves", "1 tsp. nutmeg", "1/4 c. chopped red pepper", "1 mango, pureed", "1 c. brown sugar", "raisins", "1/2 tsp. cinnamon", "1/2 tsp. white pepper"]
Directions: ["Combine all ingredients.", "Baste on ham last 45 minutes of baking, basting frequently."]
Link: www.cookbooks.com/Recipe-Details.aspx?id=232636

Title: Homemade Hot Chocolate
Ingredients: ["2 lb. chocolate drink mix", "8 qt. powdered milk", "2 lb. powdered sugar", "2 lb. 6 oz. jar Coffee-mate nondairy creamer"]
Directions: ["Makes a big batch.", "Excellent for Christmas.", "Place lunch baggie full of homemade hot chocolate in green or red lunch bags for gifts."]
Link: www.cookbooks.com/Recipe-Details.aspx?id=875388


# Example query 2

In [8]:
user_preferences = {
    "dietary_restrictions": ["gluten-free", "dairy-free"],
    "cuisine_preferences": ["Mexican"],
    "available_ingredients": ["avocado", "lime", "black beans", "corn"],
    "time_to_cook": 20,
    "difficulty_level": "Medium",
    "meal_type": "Snack"
}
recommendations = recommend_recipe(user_preferences)

print("Recommended Recipes:")
for index, recipe in recommendations.iterrows():
    print(f"\nTitle: {recipe['title']}")
    print(f"Ingredients: {recipe['ingredients']}")
    print(f"Directions: {recipe['directions']}")
    print(f"Link: {recipe['link']}")

Recommended Recipes:

Title: Black Bean Salad
Ingredients: ["2 Tbsp. olive oil", "2 Tbsp. fresh lime juice", "2 Tbsp. sliced scallions", "2 tsp. sugar", "1/2 tsp. salt", "1/2 tsp. chili powder", "2 (15 oz.) cans black beans, drained", "2 c. cherry tomato, halved", "1 Hass avocado, peeled, pitted and diced"]
Directions: ["Combine first 6 ingredients; set aside. Combine black beans, tomatoes and avocado. Stir dressing with bean mixture and serve. Serves 6."]
Link: www.cookbooks.com/Recipe-Details.aspx?id=6362

Title: Avocado Salad
Ingredients: ["1 avocado", "1 orange or pineapple", "mayonnaise with lemon juice"]
Directions: ["Cut the avocado into halves or in slices.", "Alternate slices of orange or pineapple with avocado.", "Plain lemon juice may be squeezed on avocado.", "Serve with French dressing or with thin mayonnaise."]
Link: www.cookbooks.com/Recipe-Details.aspx?id=403193

Title: Avocado Magnificent
Ingredients: ["1 avocado, cubed", "1 tomato, diced", "1 lime", "Garlic salt"]
Dir

# Example Query 3

In [9]:
user_preferences = {
    "dietary_restrictions": ["nut-free", "vegetarian"],
    "cuisine_preferences": ["Indian"],
    "available_ingredients": ["paneer", "tomatoes", "onions", "garlic", "ginger"],
    "time_to_cook": 40,
    "difficulty_level": "Medium",
    "meal_type": "Dinner"
}
recommendations = recommend_recipe(user_preferences)

print("Recommended Recipes:")
for index, recipe in recommendations.iterrows():
    print(f"\nTitle: {recipe['title']}")
    print(f"Ingredients: {recipe['ingredients']}")
    print(f"Directions: {recipe['directions']}")
    print(f"Link: {recipe['link']}")


Recommended Recipes:

Title: Tuna Casserole
Ingredients: ["7 1/4 oz. box macaroni and cheese dinner", "1/2 c. onion, chopped", "1/4 c. green pepper, chopped", "2 Tbsp. margarine", "1 can cream of celery soup", "1 small can tuna", "1/4 c. pimento, chopped", "1 (3 oz.) can mushrooms", "3/4 c. milk"]
Directions: ["Prepare dinner as directed, using 3/4 cup milk.", "Saute onion and green pepper in margarine. Add to prepared dinner with remaining ingredients.", "Pour in 1 1/2-quart casserole.", "Cover top with bread crumbs (optional).", "Bake at 350\u00b0 for 30 minutes."]
Link: www.cookbooks.com/Recipe-Details.aspx?id=581417

Title: Chutney Glaze
Ingredients: ["2 Indian mangoes, peeled and chopped", "1 c. honey", "1/2 to 1 c. pineapple juice", "1 tsp. ground cloves", "1 tsp. nutmeg", "1/4 c. chopped red pepper", "1 mango, pureed", "1 c. brown sugar", "raisins", "1/2 tsp. cinnamon", "1/2 tsp. white pepper"]
Directions: ["Combine all ingredients.", "Baste on ham last 45 minutes of baking, bas

# Example Query 4

In [10]:
user_preferences = {
    "dietary_restrictions": ["vegetarian"],
    "cuisine_preferences": ["Italian"],
    "available_ingredients": ["pasta", "tomatoes", "garlic", "olive oil", "basil", "parmesan"],
    "time_to_cook": 30,
    "difficulty_level": "Easy",
    "meal_type": "Lunch"
}
recommendations = recommend_recipe(user_preferences)

print("Recommended Recipes:")
for index, recipe in recommendations.iterrows():
    print(f"\nTitle: {recipe['title']}")
    print(f"Ingredients: {recipe['ingredients']}")
    print(f"Directions: {recipe['directions']}")
    print(f"Link: {recipe['link']}")


Recommended Recipes:

Title: Fresh Italian Pasta Salad
Ingredients: ["1 lb. box bow tie pasta", "2 to 3 cloves crushed garlic", "2 Tbsp. minced fresh basil", "3 to 4 plum tomatoes, chopped", "1/3 c. olive oil", "1/2 c. Parmesan cheese"]
Directions: ["Place garlic, basil, tomatoes, olive oil and Parmesan cheese together in large bowl.", "Cook pasta according to directions and drain well.", "While still hot, toss pasta with other ingredients. Cover and cool.", "Best when eaten warm."]
Link: www.cookbooks.com/Recipe-Details.aspx?id=499478

Title: 30-Minute Pasta Sauce
Ingredients: ["2 cloves garlic", "1/2 small onion", "3 Tbsp. olive oil", "1 large can chopped tomatoes", "1 Tbsp. basil", "1 tsp. sugar", "salt and pepper to taste", "1 tsp. Italian seasoning"]
Directions: ["Chop garlic and onion very fine.", "Saute in olive oil until soft.", "Add tomatoes, basil and rest of seasonings.", "Bring to a light boil.", "Reduce heat and simmer 30 minutes, stirring occasionally."]
Link: www.cookboo